#### 1. download directly
```python
>>> !wget https://raw.githubusercontent.com/<username>/<repo>/<branch>/some_directory/file.rb
```
- [docs](https://docs.github.com/en/rest/reference/repos#get-a-repository-readme-for-a-directory)

In [ ]:
# download logo
!wget https://raw.githubusercontent.com/needleworm/bhban_rpa/master/%5BPart%202%5D%20컴퓨터%20자동화%20기초/%5BChapter%205%5D%20컴퓨터!%20디자인%20작업도%20자신%20있지%201분%20안에%20모두%20처리해!/2_5_4_사진%201천장%2C%20한%20번에%20로고%20삽입하기/sample_logo.PNG

#### 2. convert path name to URL form
```python
>>> urllib.request.pathname2url({file_path})
```
- [docs](https://docs.python.org/3/library/urllib.request.html#urllib.request.pathname2url)

In [ ]:
# file_path to url

import urllib.request

file_path = "needleworm/bhban_rpa/master/[Part 2] 컴퓨터 자동화 기초/[Chapter 5] 컴퓨터! 디자인 작업도 자신 있지 1분 안에 모두 처리해!/2_5_4_사진 1천장, 한 번에 로고 삽입하기/sample_logo.PNG"

# print(file_path)
# print(urllib.request.pathname2url(file_path))
"https://raw.githubusercontent.com/" + urllib.request.pathname2url(file_path)

'https://raw.githubusercontent.com/needleworm/bhban_rpa/master/%5BPart%202%5D%20%EC%BB%B4%ED%93%A8%ED%84%B0%20%EC%9E%90%EB%8F%99%ED%99%94%20%EA%B8%B0%EC%B4%88/%5BChapter%205%5D%20%EC%BB%B4%ED%93%A8%ED%84%B0%21%20%EB%94%94%EC%9E%90%EC%9D%B8%20%EC%9E%91%EC%97%85%EB%8F%84%20%EC%9E%90%EC%8B%A0%20%EC%9E%88%EC%A7%80%201%EB%B6%84%20%EC%95%88%EC%97%90%20%EB%AA%A8%EB%91%90%20%EC%B2%98%EB%A6%AC%ED%95%B4%21/2_5_4_%EC%82%AC%EC%A7%84%201%EC%B2%9C%EC%9E%A5%2C%20%ED%95%9C%20%EB%B2%88%EC%97%90%20%EB%A1%9C%EA%B3%A0%20%EC%82%BD%EC%9E%85%ED%95%98%EA%B8%B0/sample_logo.PNG'

# Rrepare

In [ ]:
%%writefile noise.py
#!usr/bin/bash python
#-*-coding:utf-8

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.12.12
"""

import time
import os
import numpy as np
from PIL import Image
import shutil

print("Process Start.")
start_time = time.time()

NUM_SAMPLES = 1000

out_dir = "random_image"
if out_dir not in os.listdir():
    os.mkdir(out_dir)
else:
    shutil.rmtree(out_dir)
    os.mkdir(out_dir)

for i in range(NUM_SAMPLES):
    name = str(time.time())[-7:] + ".png"
    
    Xdim, Ydim = np.random.randint(100, 400, size=2)

    # revised
    image = np.random.randint(256, size=(Xdim, Ydim, 3)).astype('uint8')

    result = Image.fromarray(image)
    result.save(out_dir + "/" + name)

    result.close()

print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Writing noise.py


In [ ]:
%run noise.py

Process Start.
Process Done.
Peter's code 14.910093545913696 seconds.


# RPA design code
- 목표 정하기
  1. 로고 파일을 불러오기
  1. 로고를 삽입할 파일을 불러오기
  1. 로고를 삽입할 이미지 크기에 어울리는 크기로 로고 사이즈 변경하기
  1. 로고를 적당한 위치에 삽입하기
  1. 로고의 투명도 유지하기

- 목표를 달성하는 데 필요한 작업 쪼개기
  1. 로고의 크기를 그때그때 알맞게 변경하기
  1. 로고의 삽입 위치를 그때그때 알맞게 변경하기
  1. 로고 파일의 투명도를 유지하며 로고를 삽입하기

#### 1. 로고의 크기를 변경하기
- 로고는 보통 원본 이미지의 가로와 세로 중 짧은 축의 20% 정도 크기가 적당하다.
- ```image.resize()``` 함수를 이용한다.

#### 2. 로고의 삽입 위치를 변경하기
- 로고의 위치는 대충 왼쪽 상단에 여백을 살짝 띄우고 삽입해야 이쁘다. 대충 여백 2% 가량 주자.

#### 3. 로고 파일의 투명도를 유지하며 삽입하자.
- ```image.paste()``` 함수에서 인자하나를 추가하자
```python
>>> image_A.paste(image_B, (x_offset, y_offset), image_B)
```

In [ ]:
%%writefile insert_logo.py

"""
Peter's Code
Clone Code w/ 6개월 치 업무를 하루만에 끝내는 업무 자동화
Last Modification : 2021.12.16
"""

import time
import os
from PIL import Image
import sys

print("Process Start.")
start_time = time.time()

directory = sys.argv[1]
logo_filename = sys.argv[2]

out_dir = "images_with_logo"
if out_dir not in os.listdir():
    os.mkdir(out_dir)

input_files = os.listdir(directory)

logo = Image.open(logo_filename)
logo_x, logo_y = logo.size

for filename in input_files:
    # exception
    exp = filename.strip().split('.')[-1]
    if exp not in "JPG JPEG PNG BMP jpg jpeg png bmp":
        continue
    
    image = Image.open(directory + "/" + filename)

    Xdim, Ydim = image.size

    if logo_x / Xdim > logo_y / Ydim:
        new_logo_x = int(Xdim/5)
        # calculate proportional expression: new_logo_y : logo_y = new_logo_x : logo_x
        new_logo_y = int(logo_y * (new_logo_x / logo_x))

    else:
        new_logo_y = int(Ydim / 5)
        new_logo_x = int(logo_x * (new_logo_y / logo_y))

    resized_logo = logo.resize((new_logo_x, new_logo_y))
    # 3rd positional argument: mask
    image.paste(resized_logo, (int(Xdim/50), int(Ydim/50)), resized_logo)

    image.save(out_dir + "/" + filename)
    image.close()

print("Process Done.")
end_time = time.time()
print("Peter's code " + str(end_time - start_time) + " seconds.")

Overwriting insert_logo.py


In [ ]:
%run insert_logo.py random_image sample_logo.PNG

Process Start.
Process Done.
Peter's code 38.33765506744385 seconds.
